# Linear Regression Practice

For this exercise, you've been provided with a csv file, gdp_le.csv, which contains the gdp per capita and life expectancy values that you were working with on the UN data exercise.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [2]:
gdp_le = pd.read_csv('../data/gdp_le.csv')
gdp_le

,Country,Year,GDP_Per_Capita,Continent,Life_Expectancy
0,Afghanistan,2021,1517.016266,Asia,61.982
1,Afghanistan,2020,1968.341002,Asia,62.575
2,Afghanistan,2019,2079.921861,Asia,63.565
3,Afghanistan,2018,2060.698973,Asia,63.081
4,Afghanistan,2017,2096.093111,Asia,63.016
...,...,...,...,...,...
5499,Zimbabwe,1994,2670.106615,Africa,52.588
5500,Zimbabwe,1993,2458.783255,Africa,54.426
5501,Zimbabwe,1992,2468.278257,Africa,56.435
5502,Zimbabwe,1991,2781.787843,Africa,58.091


Start by fitting a linear regression model with target being life expectancy and predictor variable year.

In [6]:
lr_gdp_le = smf.ols("Life_Expectancy ~ Year", data=gdp_le).fit()
lr_gdp_le.params

Intercept   -468.138140
Year           0.267722
dtype: float64

a. What coefficients do you get? Interpret the meaning of these coefficents.

Our increase will be 0.2677(Year) starting at -468.138 on the Life Expectancy.

b. Using just the year makes the intercept term difficult to interpret. Create a new model, but this time use years since 1990 as your predictor variable. Hint: You can the patsy identity function to modify your predictors in your model formula. Inspect the new coefficients and interpret the meaning of them. Are they statistically significant?

In [26]:
gdp_le['year_since_1990']= gdp_le['Year']-1990
#gdp_le.plot(kind="scatter", x="Life_Expectancy", y="Year")

In [28]:
lr_gdp_le = smf.ols("Life_Expectancy ~ Year", data=gdp_le).fit()
lr_gdp_le.params

#make this work

Intercept   -468.138140
Year           0.267722
dtype: float64

In [ ]:
y ~ a + a:b + np.log(x)

c. Compare the actual mean life expectancy per year to the model's estimate. How well does it do?

d. Plot the actual values against your model's estimates for the mean life expectancy.

e. Inspect the R-squared value for the model. does it make sense, given the plot?

Filter the full dataset down to just the year 2021. Fit a linear regression model with target being life expectancy and predictor variable gdp per capita.

a. What coefficients do you get? Interpret the meaning of those coefficients.

b. Refit your model, but this time use thousands of dollars of gdp per capita as your predictor. How does this change your coefficients?

c. Are the coefficients statistically significant?

d. What does your model estimate for the mean life expectancy for a country whose gdp per capita is $50,000? What about one whose gdp per capita is $100,000?

e. Plot the actual values compared to your model's estimates for mean life expectancy. How would you assess the model's fit?

Now, fit a model for life expectancy based on the log of gdp per capita.

a. Inspect the coefficients for this model. Are they statistically significant? 

b. Interpret these coefficients. What does the model estimate for the average life expectancy for countries with a gdp per capita of $50,000? What about for those with a gdp per capita of $100,000? 

c. Plot the actual values compared to your models' estimates for the mean life expectancy. How does this compare to the non-logged model?

Finally, return to the full dataset.

a. First, fit a linear regression model for life expectancy based on the log of gdp per capita. 

b. Then, add the year variable to your model. How can you interpret the coefficient associated with year? What limitations or weaknesses might this model have?